# Fast Agent - Build a SQL Agent fast!

<img src="./assets/LC_L1_top.png" align="left" width="500">

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env, doublecheck_pkgs

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")  # check environmental variables
doublecheck_pkgs(pyproject_path="pyproject.toml", verbose=True)   # check packages

OPENAI_API_KEY=****here
LANGSMITH_API_KEY=****754b
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials
Python 3.12.3 satisfies requires-python: >=3.11,<3.14
package                | required | installed | status | path                                                                           
---------------------- | -------- | --------- | ------ | -------------------------------------------------------------------------------
langgraph              | >=1.0.0  | 1.0.1     | ✅ OK   | /home/abdul/langchain/2-langchain_essentials/.venv/lib/python3.12/site-packages
langchain              | >=1.0.0  | 1.0.2     | ✅ OK   | /home/abdul/langchain/2-langchain_essentials/.venv/lib/python3.12/site-packages
langchain-core         | >=1.0.0  | 1.0.1     | ✅ OK   | /home/abdul/langchain/2-langchain_essentials/.venv/lib/python3.12/site-packages
langchain-openai       | >=1.0.0  | 1.0.1     | ✅ OK   | /home/abdul/langchain/2-langchain_essentials/.venv/lib/python3.12/site-packages
langchain-anthropic    

Define the runtime context to provide the agent and tools with access to the database.

In [2]:
from dataclasses import dataclass

from langchain_community.utilities import SQLDatabase


# define context structure to support dependency injection
@dataclass
class RuntimeContext:
    db: SQLDatabase

<b>⚠️ Security Note:</b> This demo does not include a filter on LLM-generated commands. In production, you would want to limit the scope of LLM-generated commands. ⚠️   
This tool will connect to the database. Note the use of `get_runtime` to access the graph **runtime context**.

Add a system prompt to define your agents behavior.

In [3]:
import json
import operator
from typing import Annotated, List, TypedDict, Union
from langchain_community.utilities import SQLDatabase
from langchain_ollama import ChatOllama
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, ToolMessage, AIMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig

# --- 1. DATABASE SETUP ---
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

# --- 2. STATE DEFINITION ---
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

# --- 3. TOOLS DEFINITION ---
@tool
def get_schema(table_name: str, config: RunnableConfig) -> str:
    """Get the column names and types for a specific table. 
    Use this if a query fails or if you are unsure of schema."""
    database = config["configurable"].get("db")
    try:
        return database.get_table_info([table_name])
    except Exception as e:
        return f"Error: {e}"

@tool
def execute_sql(query: str, config: RunnableConfig) -> str:
    """Execute a SQLite command and return results."""
    database = config["configurable"].get("db")
    try:
        # Run and return as string for the LLM to read
        return str(database.run(query))
    except Exception as e:
        return f"Error: {e}"

# --- 4. MODEL & PARSING LOGIC ---
llm = ChatOllama(model="qwen2.5-coder:7b", temperature=0)

SYSTEM_PROMPT = """You are a SQL analyst for the Chinook database. 

CRITICAL RULES:
1. SQLite does not allow dynamic table names in COUNT queries. 
2. FIRST: Call `execute_sql` with "SELECT name FROM sqlite_master WHERE type='table';" to see all tables.
3. SECOND: Once you have the table names, write a query using UNION ALL or individual counts to find the largest one.
4. If the solution is table level, ensure that all the columns in all the tables are checked to identify the correct column.
5. If you get an 'Error', do NOT repeat the same query. Try a different approach.
6. Provide the numerical value supporting the selection.
7. Provide answers in JSON tool-call format.
"""

def call_model(state: AgentState):
    messages = [SystemMessage(content=SYSTEM_PROMPT)] + state["messages"]
    # Bind tool for standard support
    llm_with_tools = llm.bind_tools([execute_sql])
    response = llm_with_tools.invoke(messages)
    
    # --- MANUAL PARSING FIX FOR OLLAMA ---
    # If the model wrote JSON in the text but forgot the tool_calls field:
    if not response.tool_calls and "{" in response.content:
        try:
            # Extract JSON from the content
            start = response.content.find("{")
            end = response.content.rfind("}") + 1
            json_str = response.content[start:end]
            tool_data = json.loads(json_str)
            
            # Manually inject the tool call into the response object
            response.tool_calls = [{
                "name": tool_data["name"],
                "args": tool_data["arguments"],
                "id": "manual_call_" + str(len(state["messages"]))
            }]
        except:
            pass # Not valid JSON, proceed as normal text
            
    return {"messages": [response]}

# --- 5. GRAPH CONSTRUCTION ---
workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", ToolNode([execute_sql]))

workflow.add_edge(START, "agent")
# tools_condition now works because we manually populated tool_calls
workflow.add_conditional_edges("agent", tools_condition)
workflow.add_edge("tools", "agent")

memory = InMemorySaver()
app = workflow.compile(checkpointer=memory)

# --- 6. EXECUTION ---
config = {
    "configurable": {
        "thread_id": "L1",
        "db": db 
    }
}

inputs = {"messages": [HumanMessage(content="Which table has the largest number of entries?")]}

print("Starting SQL Analysis...\n")
for event in app.stream(inputs, config=config, stream_mode="values"):
    if "messages" in event:
        # This will now show the Tool Call, then the Tool Result, then the Final Answer
        event["messages"][-1].pretty_print()

Starting SQL Analysis...

================================ Human Message =================================

Which table has the largest number of entries?
================================== Ai Message ==================================

```json
{
  "name": "execute_sql",
  "arguments": {
    "query": "SELECT name FROM sqlite_master WHERE type='table';"
  }
}
```
Tool Calls:
  execute_sql (manual_call_1)
 Call ID: manual_call_1
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================

```json
{
  "name": "execute_sql",
  "arguments": {
    "query": "SELECT 'Album' AS table_name, COUNT(*) AS count FROM Album UNION ALL SELECT 'Artist' AS 

In [4]:
# --- 7. EXECUTION ---
config = {
    "configurable": {
        "thread_id": "L1",
        "db": db 
    }
}

inputs = {"messages": [HumanMessage(content="Which genre on average has the longest tracks?")]}

print("Starting SQL Analysis...\n")
for event in app.stream(inputs, config=config, stream_mode="values"):
    if "messages" in event:
        # This will now show the Tool Call, then the Tool Result, then the Final Answer
        event["messages"][-1].pretty_print()

Starting SQL Analysis...

================================ Human Message =================================

Which genre on average has the longest tracks?
================================== Ai Message ==================================

```json
{
  "name": "execute_sql",
  "arguments": {
    "query": "SELECT T2.Name AS GenreName, AVG(T3.Milliseconds) AS AverageTrackLength FROM GENRE AS T1 JOIN TRACK AS T3 ON T1.GenreId = T3.GenreId GROUP BY T1.Name ORDER BY AverageTrackLength DESC LIMIT 1;"
  }
}
```
Tool Calls:
  execute_sql (manual_call_9)
 Call ID: manual_call_9
  Args:
    query: SELECT T2.Name AS GenreName, AVG(T3.Milliseconds) AS AverageTrackLength FROM GENRE AS T1 JOIN TRACK AS T3 ON T1.GenreId = T3.GenreId GROUP BY T1.Name ORDER BY AverageTrackLength DESC LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such column: T2.Name
[SQL: SELECT T2.Name AS GenreName, AVG(T3.Milliseconds) AS 